In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data import get_datasets, get_dataloaders
from random_texts import CLIPZeroShotClassifier

In [3]:
datasets, classnames = get_datasets(fraction=1e-3)
for name, dataset in datasets.items():
    print(len(dataset))

Loading dataset shards:   0%|          | 0/27 [00:00<?, ?it/s]

120
15
21
51
52
20


In [4]:
baseline_model = CLIPZeroShotClassifier(classnames)
dataloaders = get_dataloaders(datasets, baseline_model.preprocess, batch_size=128)

100%|██████████| 345/345 [00:23<00:00, 14.48it/s]


In [ ]:
from torch import nn
import torch
from tqdm import tqdm
from pprint import pprint


def evaluate(model: nn.Module) -> dict[str, float]:
    model.eval()
    results = {}
    with torch.inference_mode():
        for name, dataloader in dataloaders.items():
            correct = 0
            total = 0
            for batch in tqdm(dataloader, desc=f"Evaluating {name}"):
                images = batch["image"]
                labels = batch["label"]
                logits = model(images)
                correct += (logits.argmax(dim=-1) == labels).float().sum()
                total += len(labels)
            results[name] = correct / total
    return results

## Zero-shot model

In [6]:
baseline_results = evaluate(baseline_model)
pprint(baseline_results)

Evaluating OOD_clipart: 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]

{'ID': tensor(0.8417),
 'OOD_clipart': tensor(0.6500),
 'OOD_infograph': tensor(0.4000),
 'OOD_painting': tensor(0.7619),
 'OOD_quickdraw': tensor(0.1373),
 'OOD_real': tensor(0.8269)}


## Full fine-tuning

In [7]:
ft_model = CLIPZeroShotClassifier(classnames)

100%|██████████| 345/345 [00:25<00:00, 13.67it/s]


In [8]:
optimizer = torch.optim.AdamW(ft_model.parameters(), lr=3e-5, weight_decay=0.1)
total_steps = len(dataloaders["ID"])
warmup_steps = min(500, total_steps // 2)
warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=0.1,
    total_iters=warmup_steps,
)
cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=total_steps - warmup_steps,
)
scheduler = torch.optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers=[warmup_scheduler, cosine_scheduler],
    milestones=[warmup_steps],
)
pbar = tqdm(dataloaders["ID"], desc="Fine-tuning")
for batch in pbar:
    images = batch["image"]
    labels = batch["label"]
    logits = ft_model(images)
    loss = nn.functional.cross_entropy(logits, labels)
    pbar.set_postfix(loss=f"{loss.item():.4f}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Fine-tuning: 100%|██████████| 1/1 [00:25<00:00, 25.70s/it, loss=0.6562]


In [9]:
ft_results = evaluate(ft_model)
pprint(ft_results)

Evaluating OOD_clipart: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it]

{'ID': tensor(0.9083),
 'OOD_clipart': tensor(0.6500),
 'OOD_infograph': tensor(0.4000),
 'OOD_painting': tensor(0.7619),
 'OOD_quickdraw': tensor(0.1373),
 'OOD_real': tensor(0.8269)}
